In [1]:
import warnings

In [3]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import pandas as pd
import numpy as np
from custom_library.custom_lib import *

plt.style.use(style='seaborn')
%matplotlib inline
sns.set()

# Import data set:

In [ ]:
pd.options.mode.chained_assignment = None


df_full = import_dataset(encoding='utf-8')
# Convert json to csv:
# df_full.to_csv('../data/dk_asylum_data_1.csv', encoding='utf-8', index=False)
df_full.shape

(9359, 5)

# Dataset preprocessing:

Keep ALL cases except for Unknown Homeland (Ukendt hjemland):

In [ ]:
df_ALL = df_full[df_full.country != "Ukendt hjemland"]

# Drop redundant column:
df_ALL.drop('hasText', axis=1, inplace=True)

df = df_ALL

df["text"] = df["text"].astype("string")

df.shape

(9167, 4)

In [ ]:

df.head()

,id,asylum_motive,country,text
0,0,"[Generelle forhold, Privatretlig forhold, Æn...",Afghanistan,Nævnet stadfæstede i februar 2022 en g...
1,1,"[Sur Place, Helbredsmæssige forhold, Generel...",Afghanistan,Nævnet stadfæstede i februar 2022 en g...
2,2,"[Internal Flight Alternative, Generelle forho...",Afghanistan,ævnet meddelte i marts 2022 opholdstil...
3,3,"[Sur Place, Chikane, Generelle forhold, Øvr...",Afghanistan,Nævnet meddelte i februar 2022 opholds...
4,4,"[Sur Place, Chikane, Øvrige modsætningsforho...",Afghanistan,Nævnet meddelte i februar 2022 opholds...


Check for potential duplicates and drop them:

In [ ]:
duplicates_df = df[df.duplicated(['text'], keep='last')]
#pd.concat(g for _, g in df.groupby("text") if len(g) > 1)

# See all duplicates:
duplicates_df

,id,asylum_motive,country,text
69,69,"[Generelle forhold, Øvrige modsætningsforhold...",Syrien,Nævnet omgjorde i november 2021 Udlænd...
70,70,"[Generelle forhold, Øvrige modsætningsforhold...",Syrien,Nævnet omgjorde i november 2021 Udlænd...
457,457,"[Udsendelse (§ 49 a m.v.), Sur Place, Privat...",Afghanistan,Nævnet stadfæstede i juli 2021 Udlændi...
697,697,"[Første asylland/§29 b, Generelle forhold]",Italien,Nævnet stadfæstede i februar 2021 Udlæ...
701,701,"[Inddragelse/forlængelse, Militære forhold, ...",Syrien,Nævnet omgjorde i marts 2021 Udlænding...
...,...,...,...,...
9128,9128,"[Politiske forhold, Agents of Persecution, T...",Afghanistan,Nævnet meddelte i juli 2010 opholdstilladelse ...
9309,9309,[Politiske forhold],Afghanistan,Nævnet stadfæstede i juni 2005 Udlændi...
9320,9320,[Privatretlig forhold],Afghanistan,Nævnet stadfæstede i december 2005 Udl...
9328,9328,[Politiske forhold],Afghanistan,Nævnet stadfæstede i april 2004 en afg...


See shape of new data frame:

In [ ]:
# Drop duplicates from original dataframe based on text column
# If the summary is the same probalbly were are looking at a duplicate
# without taking into account all other variables(columns):
df = df.drop_duplicates('text', keep='last')
# Check shape again:
df.shape

(8689, 4)